In [1]:
import os
import random
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from tsfresh.feature_extraction import settings

In [4]:
def load_data(file_path):
    """
    Reads all data files (metadata and signal matrix data) as python dictionary,
    the pkl and csv files must have the same file name.

    Arguments:
    file_path -- {str} -- path to the iq_matrix file and metadata file

    Returns:
    Python dictionary
    """
    pkl = load_pkl_data(file_path)
    meta = load_csv_metadata(file_path)
    data_dictionary = {**meta, **pkl}

    for key in data_dictionary.keys():
        data_dictionary[key] = np.array(data_dictionary[key])

    return data_dictionary
  


def load_pkl_data(file_path):
    """
    Reads pickle file as a python dictionary (only Signal data).

    Arguments:
    file_path -- {str} -- path to pickle iq_matrix file

    Returns:
    Python dictionary
    """
    path = os.path.join(mount_path, competition_path, file_path + '.pkl')
    with open(path, 'rb') as data:
        output = pickle.load(data)
    return output


def load_csv_metadata(file_path):
    """
    Reads csv as pandas DataFrame (only Metadata).

    Arguments:
    file_path -- {str} -- path to csv metadata file

    Returns:
    Pandas DataFarme
    """
    path = os.path.join(mount_path, competition_path, file_path + '.csv')
    with open(path, 'rb') as data:
        output = pd.read_csv(data)
    return output

In [5]:
# Functions for preprocessing and preprocess function
def fft(iq, axis=0):
    """
    Computes the log of discrete Fourier Transform (DFT).

    Arguments:
    iq_burst -- {ndarray} -- 'iq_sweep_burst' array
    axis -- {int} -- axis to perform fft in (Default = 0)

    Returns:
    log of DFT on iq_burst array
    """
    iq = np.log(np.abs(np.fft.fft(hann(iq), axis=axis)))
    return iq


def hann(iq, window=None):
    """
    Preformes Hann smoothing of 'iq_sweep_burst'.

    Arguments:
      iq {ndarray} -- 'iq_sweep_burst' array
      window -{range} -- range of hann window indices (Default=None)
               if None the whole column is taken

    Returns:
      Regulazied iq in shape - (window[1] - window[0] - 2, iq.shape[1])
    """
    if window is None:
        window = [0, len(iq)]

    N = window[1] - window[0] - 1
    n = np.arange(window[0], window[1])
    n = n.reshape(len(n), 1)
    hannCol = 0.5 * (1 - np.cos(2 * np.pi * (n / N)))
    return (hannCol * iq[window[0]:window[1]])[1:-1]


def max_value_on_doppler(iq, doppler_burst):
    """
    Set max value on I/Q matrix using doppler burst vector. 

    Arguments:
    iq_burst -- {ndarray} -- 'iq_sweep_burst' array
    doppler_burst -- {ndarray} -- 'doppler_burst' array (center of mass)

    Returns:
    I/Q matrix with the max value instead of the original values
    The doppler burst marks the matrix values to change by max value
    """
    iq_max_value = np.max(iq)
    for i in range(iq.shape[1]):
        if doppler_burst[i]>=len(iq):
            continue
        iq[doppler_burst[i], i] = iq_max_value
    return iq


def normalize(iq):
    """
    Calculates normalized values for iq_sweep_burst matrix:
    (vlaue-mean)/std.
    """
    m = iq.mean()
    s = iq.std()
    return (iq-m)/s


def data_preprocess(data, function=fft, feature_name='iq_x_spectrogram'):
    """
    Preforms data preprocessing.
    Change target_type lables from string to integer:
    'human'  --> 1
    'animal' --> 0

    Arguments:
    data -- {ndarray} -- the data set

    Returns:
    processed data (max values by doppler burst, DFT, normalization)
    """
    X=[]
    for i in range(len(data['iq_sweep_burst'])):
        iq = function(data['iq_sweep_burst'][i])
        # iq = max_value_on_doppler(iq,data['doppler_burst'][i])
        iq = normalize(iq)
        X.append(iq)

    data[feature_name] = np.array(X)
    if 'target_type' in data:
        data['target_type'][data['target_type'] == 'animal'] = 0
        data['target_type'][data['target_type'] == 'human'] = 1
    return data

def fft_xy(iq):
    iq = np.fft.fft(hann(iq), axis=0)
    iq = np.fft.fft(iq, axis=1)
    iq = np.abs(iq)
    return iq

In [6]:
mount_path = '/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge'
competition_path = 'data'

In [11]:
import torch.nn as nn
import torch

m = nn.AdaptiveAvgPool2d((5,7))
input = torch.randn(1, 64, 8, 9)
output = m(input)
print(output.size())

torch.Size([1, 64, 5, 7])


# Train

In [22]:
train_path = 'MAFAT RADAR Challenge - Training Set V1'
train_df = load_csv_metadata(train_path)
train_pkl = load_pkl_data(train_path)
train_data = load_data(train_path)
train_data = data_preprocess(train_data.copy())

In [ ]:
train_pkl['iq_x_spectrogram'] = train_data['iq_x_spectrogram']

In [ ]:
train_data = data_preprocess(train_data.copy(), function=fft_xy, feature_name='iq_xy_spectrogram')
train_pkl['iq_xy_spectrogram'] = train_data['iq_xy_spectrogram']

In [ ]:
train_df["target_type"] = train_data["target_type"]

# Features

In [37]:
import tsfresh
from tsfresh import extract_features
from tsfresh.feature_extraction import settings

In [77]:
train_csv = pd.read_csv("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_train_v5.csv")
val_csv = pd.read_csv("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_val_v5.csv")

In [5]:
with open("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/MAFAT RADAR Challenge - Training Set V1.pkl", "rb") as f_hdl:
    original_data = pickle.load(f_hdl)

In [6]:
with open("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/MAFAT RADAR Challenge - Auxiliary Experiment Set V2.pkl", "rb") as f_hdl:
    aux_data = pickle.load(f_hdl)

In [7]:
with open("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/MAFAT RADAR Challenge - Auxiliary Synthetic Set V2.pkl", "rb") as f_hdl:
    synth_data = pickle.load(f_hdl)

In [8]:
original_data.pop("iq_sweep_burst")
aux_data.pop("iq_sweep_burst")
synth_data.pop("iq_sweep_burst")
synth_data["segment_id"] = [x - 2000000 for x in synth_data["segment_id"]]

In [9]:
all_doppler_burst = np.concatenate([original_data["doppler_burst"], aux_data["doppler_burst"], synth_data["doppler_burst"]])
all_segment_id =  np.concatenate([original_data["segment_id"], aux_data["segment_id"], synth_data["segment_id"]])

In [10]:
unique_segments = set(train_csv.segment_id)
train_idx = [idx for idx, x in enumerate(all_segment_id) if x in unique_segments]
val_idx = [idx for idx, x in enumerate(all_segment_id) if x not in unique_segments]

In [11]:
train_doppler_burst = all_doppler_burst[train_idx]
val_doppler_burst = all_doppler_burst[val_idx]

In [12]:
rows, cols = train_doppler_burst.shape
train_ts_df = pd.DataFrame(data=train_doppler_burst.ravel(), columns=["burst"])
train_ts_df["time"] = list(range(cols))*rows
train_ts_df["id"] = np.repeat(list(range(rows)), cols).tolist()

In [13]:
rows, cols = val_doppler_burst.shape
val_ts_df = pd.DataFrame(data=val_doppler_burst.ravel(), columns=["burst"])
val_ts_df["time"] = list(range(cols))*rows
val_ts_df["id"] = np.repeat(list(range(rows)), cols).tolist()

In [14]:
train_ts_df_groups = list(train_ts_df.groupby("id"))

In [16]:
param_settings = settings.ComprehensiveFCParameters()
train_list = list()
for x in np.split(np.array(range(len(train_ts_df_groups))), 5):
    aux_df = pd.concat([train_ts_df_groups[idx][1] for idx in x])
    features = extract_features(aux_df, column_id="id", column_sort="time", default_fc_parameters=param_settings)
    train_list.append(features)
val_extracted_features = extract_features(val_ts_df, column_id="id", column_sort="time", default_fc_parameters=param_settings)

Feature Extraction: 100%|██████████| 40/40 [00:03<00:00, 11.53it/s]


In [17]:
train_extracted_features = pd.concat(train_list)

In [78]:
train_csv = train_csv.join(train_extracted_features)
val_csv = val_csv.join(val_extracted_features)

In [79]:
print(val_csv.shape)
print(train_csv.shape)

(515, 772)
(106095, 772)


In [80]:
all_columns = set(train_csv.columns)
val_null_columns = set(val_csv.loc[:, val_csv.isnull().any(axis=0)].columns)
train_null_columns = set(train_csv.loc[:, train_csv.isnull().any(axis=0)].columns)
null_columns = train_null_columns.union(val_null_columns)
accept_columns = all_columns.difference(null_columns)

In [83]:
train_csv = train_csv.loc[:, accept_columns]
val_csv = val_csv.loc[:, accept_columns]

In [84]:
print(val_csv.shape)
print(train_csv.shape)

(515, 385)
(106095, 385)


In [88]:
train_csv.to_csv("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_train_v5_ts.csv", index=False)
val_csv.to_csv("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_val_v5_ts.csv", index=False)

In [91]:
len([x for x in train_csv.columns if "burst_" in x])

376

In [72]:
train_extracted_features.shape

(106095, 763)

In [ ]:
##########################################

In [ ]:
# df: shape id

In [7]:
iq_sweep_burst = np.concatenate(np.split(train_data["iq_sweep_burst"], 6656, axis=0), axis=2).reshape(128, 212992).T

In [8]:
abs_df = pd.DataFrame(np.abs(iq_sweep_burst), columns=["range_abs_{}".format(x) for x in range(128)])

In [9]:
phase_df = pd.DataFrame(np.angle(iq_sweep_burst), columns=["range_phase_{}".format(x) for x in range(128)])

In [10]:
ts_df = pd.concat([phase_df, abs_df], axis=1)

In [11]:
ts_df["time"] = list(range(32))*6656
ts_df["id"] = np.repeat(list(range(6656)), 32).tolist()

In [12]:
del phase_df
del abs_df
del iq_sweep_burst

In [16]:
feat_df_list = list()
for x in ts_df.id.unique():
    a = tsfresh.extract_features(ts_df.loc[ts_df.id == x], column_id="id", default_fc_parameters=settings.MinimalFCParameters())
    feat_df_list.append(a)


Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1430.94it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 78259.83it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1485.88it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 3901.38it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 59074.70it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 2621.09it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 15348.56it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 3548.32it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 14440.24it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1352.23it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 18116.89it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 9534.27it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 2463.87it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 14518.59it/s]


Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 2744.53it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1558.97it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 23827.61it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 37332.03it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 852.00it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1349.84it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1683.24it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1159.27it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 2397.60it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 3319.49it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1110.57it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1343.35it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 2231.59it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 159.03it/s][A

Feat

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1287.64it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 950.09it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 14428.16it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 4274.83it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 2314.46it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 7978.06it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 2064.65it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1189.08it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 10456.79it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 3268.52it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 5988.16it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 10601.81it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 5381.60it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 40508.81it/s]

Fea

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 3421.20it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 2529.90it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 938.74it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1463.21it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 6904.66it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1163.67it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 933.78it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1216.42it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 6514.81it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 4484.07it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 972.65it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 5031.91it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1386.44it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 954.81it/s]

Feature Ex

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 16224.70it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 2240.09it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 3627.44it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 43036.40it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 3127.55it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 21285.04it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 39388.13it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 3098.76it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 8347.10it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1015.73it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1441.49it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 47443.98it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 10542.75it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 2023.09it/s]



...

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 5032.40it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 2706.19it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 22313.34it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 60549.84it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1428.12it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 49502.15it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 2224.27it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 2407.34it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 2880.44it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 3801.42it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 12929.21it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 3227.73it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 15964.33it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 18001.30it/s]



Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 24555.26it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 5992.79it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 18231.82it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 2947.79it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 14245.39it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 3624.56it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 39781.91it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 144.32it/s]A

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 924.33it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1557.22it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 4469.22it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1892.04it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 2597.61it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 5578.94it/s]

Fea

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 20642.36it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 18895.56it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 24830.28it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 32719.64it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 32364.81it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 5615.47it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 74610.22it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 14040.46it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 36566.74it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 6224.00it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 21695.69it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 150.51it/s]A

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1557.91it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1180.02it/s

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 24878.05it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 6163.68it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 48315.46it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 9119.66it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 21402.46it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 48330.50it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 5594.82it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 45697.66it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 2977.99it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 40721.40it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 35326.48it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 18858.82it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 5365.97it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 11386.69it/s

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1742.07it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 12190.83it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 9892.23it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 26117.34it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 6503.61it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1480.12it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 14292.62it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 14219.28it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 11234.20it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 5812.11it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1927.84it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 914.05it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 1298.54it/s]

Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 2122.71it/s]

Fe

# Test

In [7]:
test_path = 'MAFAT RADAR Challenge - Public Test Set V1'
test_df = load_csv_metadata(test_path)
test_pkl = load_pkl_data(test_path)
test_data = load_data(test_path)
test_data = data_preprocess(test_data.copy())

In [8]:
test_pkl['iq_x_spectrogram'] = test_data['iq_x_spectrogram']

In [ ]:
test_data = data_preprocess(test_data.copy(), function=fft_xy, feature_name='iq_xy_spectrogram')
test_pkl['iq_xy_spectrogram'] = test_data['iq_xy_spectrogram']

# Save pickle & CSV

## Train

In [ ]:
train_df.to_csv("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_train_v1.csv", index=False)

In [ ]:
output_data = "/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_train_v1.pkl"
with open(output_data, 'wb') as handle:
    pickle.dump(train_pkl, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Test

In [10]:
output_data = "/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_test_v1.pkl"
with open(output_data, 'wb') as handle:
    pickle.dump(test_pkl, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
test_pkl["iq_sweep_burst"].shape

In [ ]:
test_pkl['iq_xy_spectrogram'].shape

# Replicating validation fold

Here I take all aux data, all train (which is mafat_train_v3_spectrogram.npy), and split it in train and validation. Validation set will be the same that guy of MAFAT chose, since I see it correlates with test set.

)

In [24]:
all_spectrogram = np.load("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_train_v2_spectrogram.npy")
val_spectrogram = np.load("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_val_v3_spectrogram.npy")

In [7]:
train_df = pd.read_csv("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_train_v2.csv")
val_df = pd.read_csv("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_val_v3.csv")

In [14]:
assert len(train_df) == len(all_spectrogram)

In [31]:
new_val_df = train_df[train_df.segment_id.isin(val_df.segment_id)]
val_spectrogram_idx = list(new_val_df.index)

new_train_df = train_df[~train_df.segment_id.isin(val_df.segment_id)]
train_spectrogram_idx = list(new_train_df.index)

In [32]:
np.testing.assert_array_equal(all_spectrogram[val_spectrogram_idx], val_spectrogram[:,:,:,0])

In [33]:
new_val_spectrogram = all_spectrogram[val_spectrogram_idx]
new_train_spectrogram = all_spectrogram[train_spectrogram_idx]

In [34]:
print(new_train_spectrogram.shape)
print(new_val_spectrogram.shape)

(55418, 126, 32)
(309, 126, 32)


In [37]:
output_data = "/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_train_v4_spectrogram.npy"
with open(output_data, 'wb') as handle:
    pickle.dump(new_train_spectrogram, handle, protocol=pickle.HIGHEST_PROTOCOL)

output_data = "/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_val_v4_spectrogram.npy"
with open(output_data, 'wb') as handle:
    pickle.dump(new_val_spectrogram, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [38]:
new_train_df.to_csv("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_train_v4.csv", index=False)
new_val_df.to_csv("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_val_v4.csv", index=False)

# Doppler burst metadata

### Train & val

In [8]:
train_burst = load_pkl_data("MAFAT RADAR Challenge - Training Set V1")["doppler_burst"]
aux_burst = load_pkl_data("MAFAT RADAR Challenge - Auxiliary Experiment Set V2")["doppler_burst"]
synth_burst = load_pkl_data("MAFAT RADAR Challenge - Auxiliary Synthetic Set V2")["doppler_burst"]

In [9]:
train_burst.shape

(6656, 32)

In [10]:
full_burst = np.concatenate([train_burst, aux_burst, synth_burst])

In [11]:
train_df = pd.read_csv("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/MAFAT RADAR Challenge - Training Set V1.csv")
aux_df = pd.read_csv("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/MAFAT RADAR Challenge - Auxiliary Experiment Set V2.csv")
synth_df = pd.read_csv("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/MAFAT RADAR Challenge - Auxiliary Synthetic Set V2.csv")
synth_df["segment_id"] = synth_df["segment_id"] - 2000000

In [12]:
full_df = pd.concat([train_df, aux_df, synth_df])
full_df = full_df.reset_index()

In [13]:
train_v7_df = pd.read_csv("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_train_v5.csv")
val_v7_df = pd.read_csv("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_val_v5.csv")

In [22]:
train_v7_indexes = list(full_df[full_df.segment_id.isin(train_v7_df.segment_id)].index)
val_v7_indexes = list(full_df[full_df.segment_id.isin(val_v7_df.segment_id)].index)

In [23]:
train_burst_set = full_burst[train_v7_indexes]
val_burst_set = full_burst[val_v7_indexes]

In [24]:
train_burst_df = pd.DataFrame(train_burst_set)
train_burst_df.columns = ["doppler_burst_" + str(x) for x in train_burst_df.columns]
val_burst_df = pd.DataFrame(val_burst_set)
val_burst_df.columns = ["doppler_burst_" + str(x) for x in val_burst_df.columns]

In [27]:
assert len(train_burst_df) == len(train_v7_df)
assert len(val_burst_df) == len(val_v7_df)

In [37]:
train_final_df = pd.concat([train_v7_df, train_burst_df], axis=1)
val_final_df = pd.concat([val_v7_df, val_burst_df], axis=1)

In [40]:
train_final_df.to_csv("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_train_v5_w_burst.csv", index=False)
val_final_df.to_csv("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_val_v5_w_burst.csv", index=False)

### Test

In [8]:
test_burst = load_pkl_data("MAFAT RADAR Challenge - Public Test Set V1")["doppler_burst"]

In [10]:
test_v7_df = pd.read_csv("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/MAFAT RADAR Challenge - Public Test Set V1.csv")

In [12]:
test_burst_df = pd.DataFrame(test_burst)
test_burst_df.columns = ["doppler_burst_" + str(x) for x in test_burst_df.columns]

In [15]:
test_final_df = pd.concat([test_v7_df, test_burst_df], axis=1)

In [17]:
test_final_df.to_csv("/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_test_w_burst.csv", index=False)